In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn import metrics
from sklearn import preprocessing

In [2]:
TRAIN_PATH = 'C:\\Users\\mychu\\Workspace\\ML_Paper\\data\\train.csv'
TEST_PATH = 'C:\\Users\\mychu\\Workspace\\ML_Paper\\data\\test.csv'
TEST_TARGET_PATH = 'C:\\Users\\mychu\\Workspace\\ML_Paper\\data\\testtarget.csv'

df = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH) # hedef sütunu içermiyor
df_correct = pd.read_csv(TEST_TARGET_PATH)

In [3]:
le = preprocessing.LabelEncoder()

In [4]:
# TRAIN

In [5]:
df_categorical = df.select_dtypes(include=['object'])
df_categorical = df_categorical.fillna("none")
for column in df_categorical:
    df_categorical[column] = le.fit_transform(df_categorical[column])

In [6]:
df_numerical = df.select_dtypes(include=['int64','float64'])

for column in df_numerical:
    df_numerical[column] = df_numerical[column].fillna(df_numerical[column].mean())

df_new = df_categorical.join(df_numerical)

c:\users\mychu\workspace\ml_paper\ml_env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
# TEST

In [8]:
df_test_categorical = df_test.select_dtypes(include=['object'])

df_test_categorical = df_test_categorical.fillna("none")

for column in df_test_categorical:
    df_test_categorical[column] = le.fit_transform(df_test_categorical[column])

In [9]:
df_test_numerical = df_test.select_dtypes(include=['int64','float64'])

for column in df_test_numerical:
    df_test_numerical[column] = df_test_numerical[column].fillna(df_test_numerical[column].mean())

df_test_new = df_test_categorical.join(df_test_numerical)

c:\users\mychu\workspace\ml_paper\ml_env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
corr = df.corr()
high_correlated_columns = corr[abs(corr["SalePrice"])>0.2].index
high_correlated_columns = high_correlated_columns.drop("SalePrice")

Hedef öznitelik veriden ayrıştırılır ve Y olarak saklanır. Geri kalan özniteikler "MinMaxScaler" kullanılarak
normalize edilir. Normalize edilmiş öznitelik değerleri X olarak saklanır.

In [11]:
x_train = df_new[high_correlated_columns].values
x_test = df_test_new[high_correlated_columns].values

In [12]:
y_test = df_correct["SalePrice"]
y_train = df["SalePrice"]

In [13]:
y_train_max = max(y_train.values)
y_train_min = min(y_train.values)
y_train = (y_train-y_train_min)/(y_train_max-y_train_min)

In [14]:
y_test_max = max(y_test.values)
y_test_min = min(y_test.values)
y_test = (y_test-y_test_min)/(y_test_max-y_test_min)

In [15]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

x_train = min_max_scaler.fit_transform(x_train)
x_test = min_max_scaler.fit_transform(x_test)

In [110]:
model = tf.keras.Sequential([
      
      tf.keras.layers.Dense(256, activation='relu', input_shape=[x_train.shape[1]]),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dropout(0.05),
      tf.keras.layers.Dense(32, activation='sigmoid'),
      tf.keras.layers.Dense(16, activation='sigmoid'),
      tf.keras.layers.Dense(8, activation='sigmoid'),
      tf.keras.layers.Dense(1)
])

In [111]:
optimizer = tf.keras.optimizers.RMSprop(0.003)
model.compile(loss='mean_absolute_error',
            optimizer=optimizer,
            metrics=['mean_absolute_error'])

In [112]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 256)               5888      
_________________________________________________________________
dense_47 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_15 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_50 (Dense)             (None, 16)               

In [113]:
EPOCHS = 102
history = model.fit(x_train, y_train,epochs=EPOCHS, verbose=0)

In [114]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(70)

,loss,mean_absolute_error,epoch
32,0.033288,0.033288,32
33,0.034518,0.034518,33
34,0.033567,0.033567,34
35,0.032809,0.032809,35
36,0.032590,0.032590,36
...,...,...,...
97,0.024751,0.024751,97
98,0.024391,0.024391,98
99,0.024501,0.024501,99
100,0.025730,0.025730,100


In [115]:
#Training Data

In [116]:
y_pred = model.predict(x_train)
metrics.r2_score(y_train,y_pred)

0.8839083133917774

In [117]:
#Test Data

In [118]:
y_pred = model.predict(x_test)
i = 900
print(f"actual={y_test[i]}, prediction={y_pred[i]}")

actual=0.21954265199406853, prediction=[0.19091523]


In [119]:
metrics.r2_score(y_test,y_pred)

0.8195094974685866

In [28]:
y_pred = y_pred.reshape(y_pred.shape[0])

In [29]:
y_pred = (y_test_max-y_test_min) * y_pred + y_test_min

In [30]:
y_pred.shape

(1459,)

In [31]:
result_df = pd.DataFrame({"Id":np.linspace(y_pred.shape[0]+2,(y_pred.shape[0]+1+y_pred.shape[0]),y_pred.shape[0],dtype=int), "SalePrice":y_pred.astype(np.int)})

In [32]:
result_df.to_csv("results.csv",index=False)

In [33]:
model.save('./weights/house_prices')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./weights/house_prices\assets
